In [1]:
import re

class FileInteraction:
    def __init__(self, file_path):
        self.file_path = file_path

    def extract_value(self, search_string):
        try:
            # Read the file content
            with open(self.file_path, 'r') as file:
                lines = file.readlines()

            # Iterate through lines to find the matching string and extract the value
            for line in lines:
                if search_string in line:
                    # Use regular expression to find the value immediately after the string
                    match = re.search(rf'{re.escape(search_string)}\s+([-+]?\d*\.?\d+)', line)
                    if match:
                        value = match.group(1)  # Extract the value found
                        print(f"Value after '{search_string}': {value}")
                        return value
                    else:
                        print(f"No value found after '{search_string}'")
                        return None

        except FileNotFoundError:
            print(f"The file {self.file_path} was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    def modify(self, search_string, str_a, str_b):
        try:
            # Read the file content
            with open(self.file_path, 'r') as file:
                lines = file.readlines()

            # Flag to track if any modifications were made
            modified = False

            # Modify the specific line
            for i, line in enumerate(lines):
                if search_string in line:
                    # Replace the specified string
                    if str_a in line:
                        lines[i] = line.replace(str_a, str_b)
                        print(f"Modified line {i+1}: {lines[i].strip()}")
                        modified = True

            if modified:
                # Write the modified content back to the file
                with open(self.file_path, 'w') as file:
                    file.writelines(lines)
                
                print(f"File '{self.file_path}' has been successfully modified.")
            else:
                print(f"No modification needed for '{search_string}'.")

        except FileNotFoundError:
            print(f"The file {self.file_path} was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")
    
    def add_code(self, file_path, code_block):
        try:
            # Open the file in append mode and add the code block
            with open(file_path, 'a') as file:
                file.write("\n")  # Ensure the block starts on a new line
                file.write(code_block)
            
            print(f"Code block added successfully to '{file_path}'.")

        except FileNotFoundError:
            print(f"The file {file_path} was not found.")
        except Exception as e:
            print(f"An error occurred: {e}")

# Add code block
code_block = """+PROG ASE urs:9 $ Linear Analysis
HEAD Calculation of forces and moments
PAGE UNII 0
CTRL OPT WARP VAL 0
LC ALL
END
+PROG WING urs:9.1 $ Graphical Output
HEAD Graphical Output
PAGE UNII 0
CTRL EMPT YES         $ create empty pages if results not available
CTRL WARN (800 802 1) $ no warnings if no values found
CTRL WARN (804 808 1) $ no warnings if no values found
CTRL WARN 873         $ no warning for 2D visibility
#define SCHR=0.2
SCHH H6 0.2
#define FILL=-
#define FILLI=-
#define FILLC=-
#define SCHRI=-
#define SCHRC=-
SIZ2 SPLI PICT
SIZE -URS SC 0 SPLI  2x1 MARG NO FORM STAN
VIEW EG3
LC 1 DESI 1
LOAD TYPE ALL
LC 2 DESI 2
LOAD TYPE ALL
LC 3 DESI 3
LOAD TYPE ALL
LC 1 DESI 1
NODE TYPE SV SCHH YES
LC 2 DESI 2
NODE TYPE SV SCHH YES
LC 3 DESI 3
NODE TYPE SV SCHH YES
LC 1 DESI 1
DEFO TYPE FULL FAC DEFA LC CURR; STRU NUME 0 0; DEFO NO
LC 2 DESI 2
DEFO TYPE FULL FAC DEFA LC CURR; STRU NUME 0 0; DEFO NO
LC 3 DESI 3
DEFO TYPE FULL FAC DEFA LC CURR; STRU NUME 0 0; DEFO NO
LC 1 DESI 1
BEAM TYPE MY
LC 2 DESI 2
BEAM TYPE MY
LC 3 DESI 3
BEAM TYPE MY
LC 1 DESI 1
BEAM TYPE MZ
LC 2 DESI 2
BEAM TYPE MZ
LC 3 DESI 3
BEAM TYPE MZ
LC 1 DESI 1
BEAM TYPE MT
LC 2 DESI 2
BEAM TYPE MT
LC 3 DESI 3
BEAM TYPE MT
LC 1 DESI 1
BEAM TYPE VZ
LC 2 DESI 2
BEAM TYPE VZ
LC 3 DESI 3
BEAM TYPE VZ
LC 1 DESI 1
BEAM TYPE VY
LC 2 DESI 2
BEAM TYPE VY
LC 3 DESI 3
BEAM TYPE VY
LC 1 DESI 1
BEAM TYPE  N
LC 2 DESI 2
BEAM TYPE  N
LC 3 DESI 3
BEAM TYPE  N
END"""


In [2]:
# Example usage:
file_path = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat"
file_interaction = FileInteraction(file_path)

file_interaction.add_code(file_path, code_block)

Code block added successfully to 'C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat'.


In [4]:
# Extract value example
file_path = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat"
file_interaction = FileInteraction(file_path)
search_string = 'POIN AUTO WIDE 0 TYPE PG P'
file_interaction.extract_value(search_string)

# Modify file example
str_a = ' 57 '
str_b = ' 7 '
file_interaction.modify(search_string, str_a, str_b)

Value after 'POIN AUTO WIDE 0 TYPE PG P': 57
Modified line 118: POIN AUTO WIDE 0 TYPE PG P 7 1.728383 8
File 'C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres_exp.dat' has been successfully modified.


In [19]:
import os
import platform
from ctypes import *
from sofistik_daten import *

class CDBinteract:
    def __init__(self, dll_path="sof_cdb_w-2024.dll"):
        """
        Initializes the CDB manager with the path to the DLL library.

        :param dll_path: Path to the DLL library for CDB management.
        """
        self.myDLL = cdll.LoadLibrary(dll_path)  # Path to the DLL, not the CDB file
        self.cdbStat = None
        self.Index = None

    def open_cdb(self, cdb_file_path, cdb_index=99):
        """
        Opens the specified CDB file.

        :param cdb_file_path: Path to the CDB file.
        :param cdb_index: CDB index (default: 99).
        """
        self.Index = c_int()
        self.Index.value = self.myDLL.sof_cdb_init(cdb_file_path.encode('utf8'), cdb_index)
        self.cdbStat = c_int()
        self.cdbStat.value = self.myDLL.sof_cdb_status(self.Index.value)
        if self.cdbStat.value != 0:
            print("CDB opened successfully, CDB Status =", self.cdbStat.value)
        else:
            print(f"Error opening CDB. Status: {self.cdbStat.value}")

    def close_cdb(self):
        """
        Closes the CDB.
        """
        self.myDLL.sof_cdb_close(0)
        self.cdbStat.value = self.myDLL.sof_cdb_status(self.Index.value)
        if self.cdbStat.value == 0:
            print("CDB closed successfully, CDB Status = 0")
        else:
            print(f"Error closing CDB. Status: {self.cdbStat.value}")

    def get_displacement(self):
        """
        Get the displacement data from the CDB.
        """
        ie = c_int(0)
        RecLen = c_int(sizeof(cn_disp))

        displacement = []
        while ie.value < 2:
            ie.value = self.myDLL.sof_cdb_get(self.Index, 24, 2, byref(cn_disp), byref(RecLen), 1)
            displacement.append(cn_disp.m_ux)

            # Always read the length of record before sof_cdb_get is called
            RecLen = c_int(sizeof(cnode))

        if displacement:
            max_displacement = max(displacement)
            print(f"Max displacement: {max_displacement}")
            return max_displacement
        else:
            print("No displacement found.")
            return None

In [20]:
# Example usage:
dll_path = r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024\interfaces\64bit\sof_cdb_w-2024.dll"  # Path to the DLL
cdb_file_path = r"C:\Users\Côme Delecourt\Desktop\TESTS\Sophistik\barres.cdb"  # Path to the CDB file

cdb_handler = CDBinteract(dll_path)

# Open the CDB file
cdb_handler.open_cdb(cdb_file_path)

# Get displacement
cdb_handler.get_displacement()

# Close the CDB file
cdb_handler.close_cdb()

CDB opened successfully, CDB Status = 3
Max displacement: 0.6325802206993103
CDB closed successfully, CDB Status = 0
